In [1]:
import pandas as pd
import time
import random
import requests
from bs4 import BeautifulSoup
import re
import json
from tqdm import tqdm
from unidecode import unidecode

In [2]:
def limpia_texto(text):
    if text is None:
        return ""
    # Elimina caracteres no alfanuméricos, caracteres, puntuación, espacios extras y signos de pesos
    cleaned_text = re.sub(r'[^\w\s.]', '', text).strip()
    # Minúsculas
    cleaned_text = cleaned_text.lower()
    #Eliminar acentos
    cleaned_text = unidecode(cleaned_text)
    return cleaned_text

def limpia_moneda(text):
    if text is None:
        return ""
    #Eliminar "\n"
    cleaned_coin = re.sub(r'\n', '', text).strip()
    #Elimina comas
    cleaned_coin = re.sub(r',', '', text).strip()
    #Eliminar signo de pesos
    cleaned_coin = re.sub(r'$', '', cleaned_coin)

    return cleaned_coin

In [3]:
def limpia_datos(df):
    df = df.reset_index(drop=True)
   
    #Eliminar registros con precio 0 o nan
    df=df[df['precio']>0]
    df=df[df['precio'].notna()]
    #Eliminar registros que en oferta contengan "terreno"
    df=df[~df['oferta'].str.contains('terreno')]
    df=df[~df['oferta'].str.contains('remodelar')]
    df=df[~df['oferta'].str.contains('hectareas')]
    #Si la fuente es goodlers, sacar el promedio de precio_min y precio_max y ponerlo en precio
    #Eliminar registros con misma oferta y mismo precio
    df=df.drop_duplicates(subset=['oferta','precio','recamaras','bathrooms'],keep='first')
    #Calcular precio por metro cuadrado
    df['precio_m2'] = df['precio'] / df['mts']

    return df

In [4]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}

In [5]:
def icasas(estado, tipo="venta"):
    if tipo== "venta":
        base_url = "https://www.icasas.mx/venta/habitacionales-casas-{}-2_5_3_0_11_0/t_departamentos/p_{}"
    elif tipo== "renta":
        base_url = "https://www.icasas.mx/renta/habitacionales-casas-{}-2_5_3_0_11_0/t_departamentos/p_{}"
    else:
        raise ValueError("Tipo de propiedad no válido. Usa 'venta' o 'renta'.")
    all_data=pd.DataFrame()
    for paginas in tqdm(range(1, 101), desc=f"Scrapeando icasas en {estado}"):
        url= base_url.format(estado, paginas)
        r=requests.get(url, headers=headers)
        soup=BeautifulSoup(r.text, 'html.parser')
        resultados=soup.find_all('li', class_='serp-snippet ad featured')
        oferta, precio, superficie, recamaras, bathrooms, lat, lon = [], [], [], [], [], [], []
        for i in resultados:
            a_tag = i.find('a', class_='detail-redirection')
            oferta.append(a_tag.get_text(strip=True) if a_tag else None)
            superficie.append(i.find('span', class_='areaBuilt').get_text(strip=True) if i.find('span', class_='areaBuilt') else None)
            recamaras.append(i.find('span', class_='rooms').get_text(strip=True) if i.find('span', class_='rooms') else None)
            bathrooms.append(i.find('span', class_='bathrooms').get_text(strip=True) if i.find('span', class_='bathrooms') else None)
            lat_tag = i.find('meta', itemprop='latitude')
            lon_tag = i.find('meta', itemprop='longitude')
            lat.append(lat_tag['content'] if lat_tag else None)
            lon.append(lon_tag['content'] if lon_tag else None)
            precio.append(i.find('div', class_='price').get_text(strip=True) if i.find('div', class_='price') else None)


        #Imprimir lens de cada lista
        temp = pd.DataFrame({'oferta': oferta, 'precio':precio, 'mts': superficie, 'recamaras': recamaras, 'bathrooms': bathrooms, 'lat': lat, 'lon': lon})
        all_data = pd.concat([all_data, temp], ignore_index=True)
    all_data["fecha_consulta"] = pd.to_datetime("today")
    all_data["fuente"] = "icasas"
    all_data["oferta"] = all_data["oferta"].apply(limpia_texto)
    #Limpiar precio
    all_data["precio"] = all_data["precio"].apply(limpia_moneda)
#Eliminar todo lo que viene después de "MX"
    #all_data["precio"] = all_data["precio"].apply(lambda x: re.sub(r'MX.*', '', x))
    # Elimina texto como "(Precio a consultar)" o "destacado"
    all_data["precio"] = all_data["precio"].astype(str)
    all_data["precio"] = all_data["precio"].str.extract(r'(\d+(?:\.\d+)?)')  # Extrae solo el número# Convierte a numérico de forma segura
    all_data["precio"] = pd.to_numeric(all_data["precio"], errors="coerce")
    all_data["mts"]= all_data["mts"].astype(str)
    all_data["mts"] = all_data["mts"].str.extract(r'(\d+)')
    all_data["mts"] = pd.to_numeric(all_data["mts"], errors="coerce")
    all_data["mts"] = all_data["mts"].astype(float)
    all_data["precio"] = all_data["precio"].astype(float)

    return all_data


In [6]:
#Venta
vivi_venta=icasas("baja-california-ensenada","venta")
vivi_venta

Scrapeando icasas en baja-california-ensenada: 100%|██████████| 100/100 [01:54<00:00,  1.15s/it]


,oferta,precio,mts,recamaras,bathrooms,lat,lon,fecha_consulta,fuente
0,casa en privada de la villa el sauzal de rodr...,73363381.0,586.0,5,5,31.873194,-116.674971,2025-09-27 23:03:14.104353,icasas
1,casa en avenida heroes de baja california 647...,9181900.0,160.0,2,2,32.075278,-116.627778,2025-09-27 23:03:14.104353,icasas
2,casa en 22984 el porvenir baja california mex,10926461.0,280.0,4,3,32.070043,-116.627399,2025-09-27 23:03:14.104353,icasas
3,casa en 22766 ensenada baja california mex,4939862.2,72.0,2,1,32.018483,-116.670583,2025-09-27 23:03:14.104353,icasas
4,casa en 22766 ensenada baja california mex,6776242.2,186.0,5,4,32.018242,-116.670288,2025-09-27 23:03:14.104353,icasas
...,...,...,...,...,...,...,...,...,...
1120,casa en mision de san diego san borja residen...,658782.0,182.0,3,1,31.7851848,-116.5890129,2025-09-27 23:03:14.104353,icasas
1121,departamento en calle 11 ulbrich ensenada baj...,428250.0,45.0,2,1,31.8697564,-116.6052653,2025-09-27 23:03:14.104353,icasas
1122,casa en av. abelardo l. rodriguez costa azul ...,380000.0,210.0,2,1,31.8503521,-116.6034498,2025-09-27 23:03:14.104353,icasas
1123,casa en versalles villa del real 1ra seccion ...,345500.0,100.0,3,2,31.7958488,-116.584675,2025-09-27 23:03:14.104353,icasas


In [7]:
vivi_limpia_venta=vivi_venta.copy()
#Eliminar si oferta dice "lote" o "terreno"
vivi_limpia_venta=vivi_limpia_venta[~vivi_limpia_venta["oferta"].str.contains("lote|terreno|renta")]
#Eliminar si lat es nulo
vivi_limpia_venta=vivi_limpia_venta[vivi_limpia_venta['lat'].notna()]
#Aplicar función de limpieza
vivi_limpia_venta=limpia_datos(vivi_limpia_venta)
vivi_limpia_venta

,oferta,precio,mts,recamaras,bathrooms,lat,lon,fecha_consulta,fuente,precio_m2
0,casa en privada de la villa el sauzal de rodr...,73363381.0,586.0,5,5,31.873194,-116.674971,2025-09-27 23:03:14.104353,icasas,125193.482935
1,casa en avenida heroes de baja california 647...,9181900.0,160.0,2,2,32.075278,-116.627778,2025-09-27 23:03:14.104353,icasas,57386.875000
2,casa en 22984 el porvenir baja california mex,10926461.0,280.0,4,3,32.070043,-116.627399,2025-09-27 23:03:14.104353,icasas,39023.075000
3,casa en 22766 ensenada baja california mex,4939862.2,72.0,2,1,32.018483,-116.670583,2025-09-27 23:03:14.104353,icasas,68609.197222
4,casa en 22766 ensenada baja california mex,6776242.2,186.0,5,4,32.018242,-116.670288,2025-09-27 23:03:14.104353,icasas,36431.409677
...,...,...,...,...,...,...,...,...,...,...
1119,casa en mision de san diego san borja residen...,658782.0,182.0,3,1,31.7851848,-116.5890129,2025-09-27 23:03:14.104353,icasas,3619.681319
1120,departamento en calle 11 ulbrich ensenada baj...,428250.0,45.0,2,1,31.8697564,-116.6052653,2025-09-27 23:03:14.104353,icasas,9516.666667
1121,casa en av. abelardo l. rodriguez costa azul ...,380000.0,210.0,2,1,31.8503521,-116.6034498,2025-09-27 23:03:14.104353,icasas,1809.523810
1122,casa en versalles villa del real 1ra seccion ...,345500.0,100.0,3,2,31.7958488,-116.584675,2025-09-27 23:03:14.104353,icasas,3455.000000


In [8]:
#guardar csv
vivi_limpia_venta.to_csv("viviendas_icasas_venta.csv",index=False)